## model try


### Load libraries

In [1]:
from cv2 import cv2
import os
import tqdm
import json
import time

import pathlib
from torch.utils.data import DataLoader
import sys
sys.path.append('../')
from tqdm import tqdm
import numpy as np

from PIL import Image

#for model
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator, RPNHead, RegionProposalNetwork
from torchvision.models.detection.faster_rcnn import *
import torch
import matplotlib.pyplot as plt
from coco_eval import CocoEvaluator
from tqdm.notebook import tqdm

#DERT model
from transformers import DetrForObjectDetection


Load model state


In [3]:
from transformers import DetrConfig, DetrForObjectDetection

config = DetrConfig.from_pretrained('facebook/detr-resnet-50',revision="no_timm",num_labels=2,id2label={0:"text",1:"fracture"},
                                                             ignore_mismatched_sizes=True) 
                                                             
detr_model = DetrForObjectDetection(config)
best_model = torch.load(f'../models/DETR_Model.pt',map_location=torch.device('cpu'))
detr_model.load_state_dict(best_model['model_state_dict'])
print(best_model['best_loss'])

0.8507886904489268


Load dataset

In [6]:
import sys
sys.path.append('../')
%cd ..
from src.data.my_rpg_dataset import CocoDetection
from src.data.my_rpg_dataset import collate_fn_COCO
from transformers import DetrImageProcessor

processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
test_dataset_detr = CocoDetection(path_folder="data", processor=processor,status='test')
test_dataloader_detr = DataLoader(test_dataset_detr, collate_fn=collate_fn_COCO, batch_size=1, shuffle=False,num_workers=0)

c:\Users\spiro\OneDrive\Υπολογιστής\Thesis\Project\master_thesis_dtu
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


c:\Users\spiro\anaconda3\lib\site-packages\transformers\models\detr\image_processing_detr.py:780: FutureWarning: The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
  warnings.warn(


In [22]:
evaluator = CocoEvaluator(coco_gt=test_dataset_detr.coco, iou_types=["bbox"])

In [ ]:
# train on the GPU or on the CPU, if a GPU is not available
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

#### Visualization

In [ ]:
# colors for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

def plot_results(pil_img, scores, labels, boxes,target_labels,target_boxes,confidence=0.5):
    fig = plt.figure(figsize=(15,10))

    fig.add_subplot(1, 2, 1)

    plt.imshow(np.asarray(pil_img),cmap='gray')
    ax = plt.gca()
    colors = COLORS * 100
    id2label = id2label={0:"text",1:"fracture"}
    for label, box,c  in zip(target_labels.tolist(), target_boxes.tolist(), colors):
        x,y,w,h = tuple(box)
        ax.add_patch(plt.Rectangle((x,y),w,h,
                                   fill=False, color=c, linewidth=3))
        text = f'{model.config.id2label[label]}'
        ax.text(x, y, text, fontsize=12,
                bbox=dict(facecolor='yellow', alpha=0.3))
    plt.axis('off')
    plt.title("Target Image and Boxes")
    ########################################################################################################
    fig.add_subplot(1, 2, 2)

    plt.imshow(np.asarray(pil_img),cmap='gray')
    ax = plt.gca()
    colors = COLORS * 100
    id2label = id2label={0:"text",1:"fracture"}
    for score, label, (xmin, ymin, xmax, ymax),c  in zip(scores.tolist(), labels.tolist(), boxes.tolist(), colors):
        if score >= confidence:
            ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                    fill=False, color=c, linewidth=3))
            text = f'{model.config.id2label[label]}: {score:0.2f}'
            ax.text(xmin, ymin, text, fontsize=12,
                    bbox=dict(facecolor='yellow', alpha=0.3))
    plt.axis('off')
    plt.title("Model prediction")


    plt.show()

In [ ]:
# Set the model to evaluation mode
detr_model.eval()

# Iterate over all batches in the test loader
for idx, batch in enumerate(tqdm(test_dataloader_detr)):
    # get the inputs
    pixel_values = batch["pixel_values"].to(device)
    pixel_mask = batch["pixel_mask"].to(device)
    labels = [{k: v.to(device) for k, v in t.items()} for t in batch["labels"]] # these are in DETR format, resized + normalized

    target = labels 

    # forward pass
    with torch.no_grad():
      outputs = detr_model(pixel_values=pixel_values, pixel_mask=pixel_mask)

    # turn into a list of dictionaries (one item for each example in the batch)
    orig_target_sizes = torch.stack([target["orig_size"] for target in labels], dim=0)
    results = processor.post_process_object_detection(outputs, target_sizes=orig_target_sizes)

    # load image based on ID
    image_id = target['image_id'].item()
    image = test_dataset_detr.coco.loadImgs(image_id)[0]
    image = Image.open(os.path.join(image['file_name']))
    target_annotations = test_dataset_detr.coco.imgToAnns[image_id]


    # postprocess model outputs
    width, height = image.size
    postprocessed_outputs = processor.post_process_object_detection(outputs,
                                                                    target_sizes=[(height, width)],
                                                                    threshold=0.5)
    results = postprocessed_outputs[0]
    plot_results(image, results['scores'], results['labels'], results['boxes'],torch.tensor([target_annotations[0]['category_id']]),torch.tensor([target_annotations[0]['bbox']]))